In [1]:
import tensorflow

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import keras

Using TensorFlow backend.


In [3]:
print(1)

1


In [4]:

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping


In [5]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15613782347307817059
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 871037223321499852
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [6]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [32]:
img_width, img_height = 224, 224
train_data_dir = "/data/oxford102/train/"
validation_data_dir = "/data/oxford102/train/"
nb_train_samples = 4604
nb_validation_samples = 1094 

In [8]:
nr_categories = 102
batch_size = 16
epochs = 50

In [9]:
model = applications.mobilenetv2.MobileNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_fungi_all_p2.h5")

In [10]:
for i, layer in enumerate(model.layers):
   print(i, layer.name)

0 input_1
1 Conv1
2 bn_Conv1
3 Conv1_relu
4 expanded_conv_depthwise
5 expanded_conv_depthwise_BN
6 expanded_conv_depthwise_relu
7 expanded_conv_project
8 expanded_conv_project_BN
9 block_1_expand
10 block_1_expand_BN
11 block_1_expand_relu
12 block_1_depthwise
13 block_1_depthwise_BN
14 block_1_depthwise_relu
15 block_1_project
16 block_1_project_BN
17 block_2_expand
18 block_2_expand_BN
19 block_2_expand_relu
20 block_2_depthwise
21 block_2_depthwise_BN
22 block_2_depthwise_relu
23 block_2_project
24 block_2_project_BN
25 block_2_add
26 block_3_expand
27 block_3_expand_BN
28 block_3_expand_relu
29 block_3_depthwise
30 block_3_depthwise_BN
31 block_3_depthwise_relu
32 block_3_project
33 block_3_project_BN
34 block_4_expand
35 block_4_expand_BN
36 block_4_expand_relu
37 block_4_depthwise
38 block_4_depthwise_BN
39 block_4_depthwise_relu
40 block_4_project
41 block_4_project_BN
42 block_4_add
43 block_5_expand
44 block_5_expand_BN
45 block_5_expand_relu
46 block_5_depthwise
47 block_5_de

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (Activation)         (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
expanded_c

In [12]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
for layer in model.layers[:]:
    print(layer.name)
    layer.trainable = False

input_1
Conv1
bn_Conv1
Conv1_relu
expanded_conv_depthwise
expanded_conv_depthwise_BN
expanded_conv_depthwise_relu
expanded_conv_project
expanded_conv_project_BN
block_1_expand
block_1_expand_BN
block_1_expand_relu
block_1_depthwise
block_1_depthwise_BN
block_1_depthwise_relu
block_1_project
block_1_project_BN
block_2_expand
block_2_expand_BN
block_2_expand_relu
block_2_depthwise
block_2_depthwise_BN
block_2_depthwise_relu
block_2_project
block_2_project_BN
block_2_add
block_3_expand
block_3_expand_BN
block_3_expand_relu
block_3_depthwise
block_3_depthwise_BN
block_3_depthwise_relu
block_3_project
block_3_project_BN
block_4_expand
block_4_expand_BN
block_4_expand_relu
block_4_depthwise
block_4_depthwise_BN
block_4_depthwise_relu
block_4_project
block_4_project_BN
block_4_add
block_5_expand
block_5_expand_BN
block_5_expand_relu
block_5_depthwise
block_5_depthwise_BN
block_5_depthwise_relu
block_5_project
block_5_project_BN
block_5_add
block_6_expand
block_6_expand_BN
block_6_expand_relu


In [18]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(102, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [19]:
model_final = Model(input = model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [20]:
model_final.shape

AttributeError: 'Model' object has no attribute 'shape'

In [21]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
# for layer in model_final.layers[:12]:
#     print(layer.name)
#     layer.trainable = False

In [22]:
# model_final.summary()

In [23]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [24]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.2)

In [25]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
# img_width, img_height = 256, 256
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 4604 images belonging to 102 classes.


In [34]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 1094 images belonging to 102 classes.


In [35]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("mobilenetv2_oxford102_train_dataset.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [36]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [37]:
#batch 16

In [38]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
288/287 [==============================] - 66s 228ms/step - loss: 4.0631 - acc: 0.1830 - val_loss: 3.4879 - val_acc: 0.2093

Epoch 00001: val_acc improved from -inf to 0.20932, saving model to mobilenetv2_oxford102_train_dataset.h5
Epoch 2/50
288/287 [==============================] - 64s 222ms/step - loss: 1.8333 - acc: 0.5547 - val_loss: 2.9786 - val_acc: 0.2907

Epoch 00002: val_acc improved from 0.20932 to 0.29068, saving model to mobilenetv2_oxford102_train_dataset.h5
Epoch 3/50
288/287 [==============================] - 64s 221ms/step - loss: 0.9890 - acc: 0.7413 - val_loss: 2.7961 - val_acc: 0.3007

Epoch 00003: val_acc improved from 0.29068 to 0.30073, saving model to mobilenetv2_oxford102_train_dataset.h5
Epoch 4/50
288/287 [==============================] - 64s 221ms/step - loss: 0.5863 - acc: 0.8448 - val_loss: 2.7298 - val_acc: 0.3309

Epoch 00004: val_acc improved from 0.30073 to 0.33090, saving model to mobilenetv2_oxford102_train_dataset.h5
Epoch 5/50
288/287 

KeyboardInterrupt: 

In [ ]:
#batch 256


In [ ]:
print(10)